In [1]:
import json
import requests
import os
import time
from datetime import datetime, timedelta

In [2]:
!git clone https://github.com/Rohan-J123/PyroPredictorModel

Cloning into 'PyroPredictorModel'...
remote: Enumerating objects: 160, done.
remote: Counting objects: 100% (160/160), done.
remote: Compressing objects: 100% (45/45), done.
remote: Total 160 (delta 125), reused 145 (delta 113), pack-reused 0 (from 0)
Receiving objects: 100% (160/160), 8.77 MiB | 24.75 MiB/s, done.
Resolving deltas: 100% (125/125), done.


In [3]:
!pip install openmeteo_requests requests_cache retry-requests

import openmeteo_requests

import requests_cache
import pandas as pd
from retry_requests import retry
import numpy as np

# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.4/61.4 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 2.1 MB/s eta 0:00:00


In [31]:
def weatherData(Latitude, Longitude, StartDate, EndDate):
    url = "https://archive-api.open-meteo.com/v1/archive"
    params = {
      "latitude": Latitude,
      "longitude": Longitude,
      "start_date": StartDate,
      "end_date": EndDate,
      "hourly": ["relative_humidity_2m", "dew_point_2m", "surface_pressure", "cloud_cover", "wind_speed_10m", "soil_temperature_0_to_7cm", "soil_moisture_0_to_7cm", "direct_radiation"],
      "daily": ["weather_code", "temperature_2m_mean", "rain_sum", "wind_speed_10m_max", "et0_fao_evapotranspiration"]
    }
    responses = openmeteo.weather_api(url, params=params)

    # Process first location. Add a for-loop for multiple locations or weather models
    response = responses[0]

    # Process hourly data. The order of variables needs to be the same as requested.
    hourly = response.Hourly()

    hourly_relative_humidity_2m = np.mean(hourly.Variables(0).ValuesAsNumpy())
    hourly_dew_point_2m = np.mean(hourly.Variables(1).ValuesAsNumpy())
    hourly_surface_pressure = np.mean(hourly.Variables(2).ValuesAsNumpy())
    hourly_cloud_cover = np.mean(hourly.Variables(3).ValuesAsNumpy())
    hourly_wind_speed_10m = np.mean(hourly.Variables(4).ValuesAsNumpy())
    hourly_soil_temperature_0_to_7cm = np.mean(hourly.Variables(5).ValuesAsNumpy())
    hourly_soil_moisture_0_to_7cm = np.mean(hourly.Variables(6).ValuesAsNumpy())
    hourly_direct_radiation = np.mean(hourly.Variables(7).ValuesAsNumpy())

    weather_data = {}

    weather_data["relative_humidity_2m"] = float(hourly_relative_humidity_2m)
    weather_data["dew_point_2m"] = float(hourly_dew_point_2m)
    weather_data["surface_pressure"] = float(hourly_surface_pressure)
    weather_data["cloud_cover"] = float(hourly_cloud_cover)
    weather_data["wind_speed_10m"] = float(hourly_wind_speed_10m)
    weather_data["soil_temperature_0_to_7cm"] = float(hourly_soil_temperature_0_to_7cm)
    weather_data["soil_moisture_0_to_7cm"] = float(hourly_soil_moisture_0_to_7cm)
    weather_data["direct_radiation"] = float(hourly_direct_radiation)

    daily = response.Daily()

    daily_weather_code = np.mean(daily.Variables(0).ValuesAsNumpy())
    daily_temperature_2m_mean = np.mean(daily.Variables(1).ValuesAsNumpy())
    daily_rain_sum = np.mean(daily.Variables(2).ValuesAsNumpy())
    daily_wind_speed_10m_max = np.mean(daily.Variables(3).ValuesAsNumpy())
    daily_et0_fao_evapotranspiration = np.mean(daily.Variables(4).ValuesAsNumpy())

    weather_data["daily_weather_code"] = float(daily_weather_code)
    weather_data["daily_temperature_2m_mean"] = float(daily_temperature_2m_mean)
    weather_data["daily_rain_sum"] = float(daily_rain_sum)
    weather_data["daily_wind_speed_10m_max"] = float(daily_wind_speed_10m_max)
    weather_data["daily_et0_fao_evapotranspiration"] = float(daily_et0_fao_evapotranspiration)

    print(weather_data)

    return weather_data

In [35]:
output_dir = "/content/Non_Fire_Dataset_Weather"
os.makedirs(output_dir, exist_ok=True)

In [36]:
file_paths = ["/content/PyroPredictorModel/Non_Fire_Dataset_Elevation/non_fire_batch_1.geojson",
              "/content/PyroPredictorModel/Non_Fire_Dataset_Elevation/non_fire_batch_2.geojson",
              "/content/PyroPredictorModel/Non_Fire_Dataset_Elevation/non_fire_batch_3.geojson",
              "/content/PyroPredictorModel/Non_Fire_Dataset_Elevation/non_fire_batch_4.geojson",
              "/content/PyroPredictorModel/Non_Fire_Dataset_Elevation/non_fire_batch_5.geojson",
              "/content/PyroPredictorModel/Non_Fire_Dataset_Elevation/non_fire_batch_6.geojson",
              "/content/PyroPredictorModel/Non_Fire_Dataset_Elevation/non_fire_batch_7.geojson",
              "/content/PyroPredictorModel/Non_Fire_Dataset_Elevation/non_fire_batch_8.geojson"]

In [37]:
import random

i = 1
for file_path in file_paths:
    with open(file_path, 'r') as file:
        geo_data = json.load(file)
        output_data = []
        for feature in geo_data['features']:
            [LONGITUDE, LATITUDE] = feature['geometry']['coordinates']
            StartDate = feature['properties']['fromdate']
            time.sleep(random.uniform(0, 0.5))
            feature["Weather"] = weatherData(LATITUDE, LONGITUDE, StartDate, StartDate)
            output_data.append(feature)
        geo_data['features'] = output_data
    with open(f"/content/Non_Fire_Dataset_Weather/non_fire_batch_{i}.geojson", 'w') as out_file:
          json.dump(geo_data, out_file, indent=4)
    i += 1

{'relative_humidity_2m': 71.3903579711914, 'dew_point_2m': 18.56999969482422, 'surface_pressure': 961.498046875, 'cloud_cover': 59.5, 'wind_speed_10m': 14.301640510559082, 'soil_temperature_0_to_7cm': 26.44916534423828, 'soil_moisture_0_to_7cm': 0.2749583423137665, 'direct_radiation': 81.25, 'daily_weather_code': 51.0, 'daily_temperature_2m_mean': 24.5575008392334, 'daily_rain_sum': 0.20000000298023224, 'daily_wind_speed_10m_max': 19.25579261779785, 'daily_et0_fao_evapotranspiration': 3.656198740005493}
{'relative_humidity_2m': 37.20627975463867, 'dew_point_2m': 10.874332427978516, 'surface_pressure': 975.7279663085938, 'cloud_cover': 2.4166667461395264, 'wind_speed_10m': 9.425877571105957, 'soil_temperature_0_to_7cm': 28.963918685913086, 'soil_moisture_0_to_7cm': 0.14862500131130219, 'direct_radiation': 170.6666717529297, 'daily_weather_code': 1.0, 'daily_temperature_2m_mean': 27.84308624267578, 'daily_rain_sum': 0.0, 'daily_wind_speed_10m_max': 20.26579475402832, 'daily_et0_fao_evapo

In [38]:
!zip -r Non_Fire_Dataset_Weather.zip Non_Fire_Dataset_Weather

  adding: Non_Fire_Dataset_Weather/ (stored 0%)
  adding: Non_Fire_Dataset_Weather/non_fire_batch_1.geojson (deflated 87%)
  adding: Non_Fire_Dataset_Weather/non_fire_batch_7.geojson (deflated 87%)
  adding: Non_Fire_Dataset_Weather/non_fire_batch_6.geojson (deflated 87%)
  adding: Non_Fire_Dataset_Weather/non_fire_batch_5.geojson (deflated 87%)
  adding: Non_Fire_Dataset_Weather/non_fire_batch_8.geojson (deflated 86%)
  adding: Non_Fire_Dataset_Weather/non_fire_batch_3.geojson (deflated 87%)
  adding: Non_Fire_Dataset_Weather/non_fire_batch_2.geojson (deflated 87%)
  adding: Non_Fire_Dataset_Weather/non_fire_batch_4.geojson (deflated 87%)
